In [13]:
import numpy as np
import glob
import os
import logging
from tqdm import tqdm
from PIL import Image

In [4]:
DATA_DIR = r"D:\minowa\BloodPressureEstimation\data\processed\npy\1030\p00"
OUTPUT_DIR = r"D:\minowa\BloodPressureEstimation\data\processed\img\1030"

```
def saveDataSets(train_data, train_data_noisy, test_data, test_data_noisy):
    for i,j in enumerate(train_data):
        rescaled = ((255.0 / (j.max() - j.min())) * (j - j.min())).astype(np.uint8)
        #rescaled = np.array(j).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save('noise2ppg_v2/trainB/%s.png' %str(i))
        
    for i,j in enumerate(train_data_noisy):
        rescaled = ((155.0 / (j.max() - j.min())) * (j - j.min())+50.0).astype(np.uint8)
        #rescaled = np.array(j).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save('noise2ppg_v2/trainA/%s.png' %str(i))

    for i,j in enumerate(test_data):
        rescaled = ((255.0 / (j.max() - j.min())) * (j - j.min())).astype(np.uint8)
        #rescaled = np.array(j).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save('noise2ppg_v2/testB/%s.png' %str(i))

    for i,j in enumerate(test_data_noisy):
        rescaled = ((155.0 / (j.max() - j.min())) * (j - j.min())+50.0).astype(np.uint8)
        #rescaled = np.array(j).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save('noise2ppg_v2/testA/%s.png' %str(i))

```

In [5]:
def create_signal_images(target,count,out_path_abp,out_path_ppg):
    px = 256
    def ppg_img(sig,out_path_ppg,count):
        rescaled = (255.0*sig).astype(np.uint8)
        #rescaled = np.array(j).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save(out_path_ppg.format(count))
        return 
    def abp_img(sig,out_path_abp,count):
        abp_max = 255
        abp_min = 0
        rescaled = (255.0/(abp_max-abp_min))*(sig-abp_min).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save(out_path_abp.format(count))
        return
    for i in range(0,len(target)-px,px):
        s = target[i:i+px,:]
        abp_img(s[:,0],out_path_abp,count)
        ppg_img(s[:,1],out_path_ppg,count)
        count+=1
    return count

In [16]:
dirs = glob.glob("p*",root_dir=DATA_DIR)
print(len(dirs))
cnt = 0
for i in tqdm(range(len(dirs))):
    files = glob.glob(dirs[i]+"\*",root_dir=DATA_DIR)
    # n_files = len(files)
    os.makedirs(f"{DATA_DIR}\\abp\p00\{dirs[i]}",exist_ok=True)
    os.makedirs(f"{DATA_DIR}\\ppg\p00\{dirs[i]}",exist_ok=True)
    for file in files:
        # p000652\3488442_0030.npy
        out_path_abp = f"{DATA_DIR}\\abp\p00\{dirs[i]}"+"_{:05d}.png"
        out_path_ppg = f"{DATA_DIR}\\ppg\p00\{dirs[i]}"+"_{:05d}.png"
        sig = np.load(os.path.join(DATA_DIR,file))
        nan_locs = np.where(np.isnan(sig))[0]
        count = 0
        for n in range(len(nan_locs)-1):
            target = sig[nan_locs[n]+1:nan_locs[n+1]]
            count = create_signal_images(target,count,out_path_abp,out_path_ppg)
        

121


  0%|          | 0/121 [00:00<?, ?it/s]


OSError: cannot write mode F as PNG